# Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Convolutional functions

- Zero Padding
- Convolve window 
- Convolution forward
- Convolution backward


In [2]:
def zero_pad(X, pad):
  """
  Add zero padding to all the images in the X dataset.

  Args:
    X: (numpy array) of shape(number_samples, height, width, channels)
    pad: (int) amount of padding around each image

  Returns:
    X_paddded: padded dataset
  """

  X_padded = np.pad(X, ((0, 0), (pad, pad), (pad, pad), (0, 0)), 'constant', constant_values=0)

  return X_padded

In [3]:
def conv_step(slice_prev, W, b):
  """
  Apply one filter using parameters W and b on a single slice of
  the output activation.

  Args:
    slice_prev: slice of input data
    W: Weight parameters contained in a matrix
      shape(filter, filter, prev_channels)
    b: bias parameters contained in a matrix
      shape(1, 1, 1)

  Returns:
    Z: scalar vlue, result of convolving the sliding window
      (W, b) on a slice of the input data
  """

  # Element-wise product between slice_prev and W
  Z = np.multiply(slice_prev, W)
  Z = np.sum(Z)

  # Add bias to Z
  # To get a scalar we cat b to a float
  Z = Z + float(b)

  return Z

In [4]:
def conv_forward(A_prev, W, b, parameters):
  """
  Compute forward propagation

  Args:
    A_prev: (numpy array) output activation of the pevious layer
    W: (numpy_array) Filters weights
    b: (numpy_array) Filters biases
    parameters: (dict) with the strides and paddings
  
  Returns:
    Z: (numpy array) conv output
    cache: values needed for the backpropagation
  """

  # Dimensions from previous activation
  samples, height_prev, width_prev, channels_prev = A_prev.shape

  # Dimensions from W
  filter, filter, channels_prev, channels = W.shape

  stride = parameters['stride']
  pad = parameters['pad']

  # Compute the dimensions of the conv output
  O_height = int((height_prev - filter + 2 * pad) / stride) + 1
  O_width = int((width_prev - filter + 2 * pad) / stride) + 1

  # Initialize the output with zeros
  Z = np.zeros((samples, O_height, O_width, channels))

  # Create A_prev_pad
  A_prev_pad = zero_pad(A_prev, pad)

  for i in range(samples):
    a_prev_pad = A_prev_pad[i]

    # Vertical axis
    for h in range(O_height):
      
      # horizontal axis
      for w in range(O_widht):

        # channels
        for c in range(channels):

          # Find the corners of the current slice
          vert_start = h * stride
          vert_end = vert_start + filter
          horiz_start = w * stride
          horiz_end = horiz_start + filter

          # Using the corners define the 3D slice of a_prev_pad
          a_slice_prev = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :]

          # Convolve the 3D slice
          weights = W[:, :, :, c]
          biases = b[:, :, :, c]
          Z[i, h, w, c] = conv_step(a_slice_prev, weights, biases)

  cache = (A_prev, W, b, parameters)

  return A, cache


In [6]:
def conv_backprop(dZ, cache):
  """
  Compute backpropagation for a convolution

  Args:
    dZ: (numpy array) gradient cost with respect to the output of the conv
      layer Z
    cache: values needed for the backpropagation

  Returns:
    dA_prev: (numpy array) gradient cost with respect to the input
      of the conv layer
    dW: (numpy array) gradient cost with respect to the weights of
      the conv layer
    db: (numpy array) gradient cost with respect to the biases of
      the conv layer
  """

  A_prev, W, b, parameters = cache

  samples, height_prev, width_prev, channels_prev = A_prev.shape

  filter, filter, channels_prev, channels = W.shape

  stride = parameters['stride']
  pad = parameters['pad']

  # Dimensions from dZ shape
  samples, dz_heigth, dz_width, dz_channels = dZ.shape

  # Initialize dA_prev, dW, db with the current shapes
  dA_prev = np.zeros((samples, height_prev, width_prev, channels_prev))
  dW = np.zeros((filter, filter, channels_prev, dz_channels))
  db = np.zeros((1, 1, 1, dz_channels))

  # Pad A_prev and dA_prev
  A_prev_ad = zero_pad(A_prev, pad)
  dA_prev_pad = zero_pad(dA_prev, pad)

  for i in range(samples):

    # Select the training example
    a_prev_pad = A_prev_pad[i]
    da_prev_pad = dA_prev_pad[i]

    for h in range(dz_height):

      for w in range(dz_width):

        for c in range(dz_channels):

          # find the corners
          vert_start = h
          vert_end = vert_start + filter
          horiz_start = w
          horiz_end = horiz_start + filter

          # use the corners to define the slice
          a_slice = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :]

          # Update the parameters
          da_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :] += W[:, :, :, c] * dZ[i, h, w, c]
          dW[:, :, :, c] += a_slice * dZ[i, h, w, c]
          db[:, :, :, c] += dZ[i, h, w, c]


    # set dA_prev to the unpadded da_prev_pad
    dA_prev[i, :, :, :] = da_prev_pad[pad:-pad, pad:-pad, :]

  return dA_prev, dW, db

# Pooling functions

- Pooling forward
- Create mask 
- Distribute value
- Pooling backward

In [5]:
def pool_forward(A_prev, parameters, mode='max'):
  """
  Forward pass of the pooling layer

  Args:
    A_prev: (numpy array) output activation of the pevious layer
    parameters: (dict) with the strides and paddings
    mode: (str) the pooling mode, it could be 'max' or 'average'

  Returns:
    A: (numpy array) pooling layer output
    cache: values needed for the backpropagation
  """

  # Dimensions from previous activation
  samples, height_prev, width_prev, channels_prev = A_prev.shape

  filter = parameters['filter']
  stride = parameters['stride']

  # Output dimensions
  O_height = int(1 + (height_prev - filter) / stride)
  O_width = int(1 + (width_prev - filter) / stride)
  channels = channels_prev

  # Initialize output matrix A
  A = np.zeros((samples, O_height, O_width, channels))

  for i in range(samples):

    # Vertical axis
    for h in range(O_height):
      
      # horizontal axis
      for w in range(O_widht):

        # channels
        for c in range(channels):

          # find the corners of the current slice
          vert_start = h * stride
          vert_end = vert_start + filter
          horiz_start = w * stride
          horiz_end = horiz_start + filter

          a_prev_slice = A_prev[i, vert_start:vert_end, horiz_start:horiz_end, c]

          # Compute the pooling operation
          if mode == 'max':
            A[i, h, w, c] = np.max(a_prev_slice)
          elif mode == 'average':
            A[i, h, w, c] = np.mean(a_prev_slice)

  cache = (A_prev, cache)

  return A, cache

In [ ]:
def